# Assign MDM ID only to accounts with a reliable match to SFDC ID

In [5]:
import pandas as pd
import os


from datetime import datetime

#File locations

#=================================== Dedupped SpecDistr output file, this file contains accounts that have been deduplicated and scored by the deduplication algorithm

# use this input if you have a dedupped SpecDistr output file
#input_base_path =r"C:\Users\beste\OneDrive - Qral Group\01 Narcan\12 Ops\20231128_A1\02 Internal Controls\mastering" #update path if needed

# use this input if you have a dedupped SpecDistr output file
#raw_input_file = os.path.join(input_base_path, 'Dedupped__SpecDist Mastering Input20231128_175947.csv') # Set path containing the file with previously analyzed accounts
#Read in raw_input1_file csv file as a dataframe. This is the dedupped SpecDistr output file
#master_data= pd.read_csv(raw_input_file)

#=================================== Historical account master log book source file, this file contains the master list of accounts

#Use this input to create new Ids for new accounts

input1_base_path =r"C:\Users\beste\OneDrive - Qral Group\01 Narcan\08 Mastering\00 Logbook"

raw_input1_file = os.path.join(input1_base_path, 'Customer Mastering LogBook v0.3.xlsx') # Set path containing the file with previously analyzed accounts

#Read in raw_input1_file file as a dataframe. This is the master account list from a EXCEL workbook against which we check the new accounts
new_account_data= pd.read_excel(raw_input1_file, sheet_name='New Accounts_1205')



# Filter rows where the Link score is greater than or equal to a pre-defined match score, a value that is based on manual review of true positive matches
filtered_new_account_data = new_account_data[new_account_data['Source'] != 'ExFactory'] # Note that this approach is row filtering and keeping all columns, master_data[master_data['Column or metric of interest by which you want to subset data']]


#=================================== Historical account Analysis source file, this file contains the list of accounts that have been previously analyzed and fuzzy matched

input1_base_path =r"C:\Users\beste\OneDrive - Qral Group\01 Narcan\08 Mastering"

raw_input2_file = os.path.join(input1_base_path, 'Mastering Specialty Distributor v1.2.xlsx') # Set path containing the file with previously analyzed accounts

#Read in raw_input1_file file as a dataframe. This is the master account list from a EXCEL workbook against which we check the new accounts
new_account_data= pd.read_excel(raw_input1_file, sheet_name='2c. Fuzzy complete MDM ID list')


# how to check max MDM ID and Max Cluster ID?
#new_account_data['MDM_ID'].max()

# If needed select rows and only specific columns
#filtered_data = master_data[master_data['SpecDistr_Link Score']>= reliable_match_score][['Q_ID', 'SF_ID']] # this line is for reducing un needed columns in the output



# Generate unique account IDs
#filtered_data['MCM_ID'] = range(1, len(filtered_data)+1)  # range accepts a start and stop value. Use 1 to start numbering at 1 and use len to count up to the number of rows in the data. Add 1 because range excludes last row/stop value
filtered_new_account_data['MCM_ID'] = range(1, len(filtered_new_account_data)+1)  # range accepts a start and stop value. Use 1 to start numbering at 1 and use len to count up to the number of rows in the data. Add 1 because range excludes last row/stop value

# to define a fixed length of leading zeroes
id_length = 7


# Generate a unique set of 'Cluster ID' values
unique_cluster_ids = filtered_new_account_data['Cluster ID'].unique()

# Create a dictionary to map each unique 'Cluster ID' to a unique 'MCM_ID'
cluster_id_to_mcm_id = {cluster_id: f"EMUSHCO{i:0{id_length}d}" for i, cluster_id in enumerate(unique_cluster_ids, start=1)}

# Map the 'MCM_ID' back to the original DataFrame
filtered_new_account_data['MCM_ID'] = filtered_new_account_data['Cluster ID'].map(cluster_id_to_mcm_id)





### Re-order columns, if needed

In [6]:
# Reorder columns to make MCM ID the first column
cols= ['MCM_ID'] + [col for col in new_account_data if col != 'MCM_ID']
new_account_data = new_account_data[cols]
 
new_account_data.head()

,MCM_ID,Cluster ID,confidence_score,Record Number,Q_ID,Transaction Bucket,Source,ShipTo Code,Name,Address,City,State,Zip,Count of Records,Max of Date,Sum of Quantity
0,EMUSHCO0000001,292,1.000000,0,Diamond | NMSM | SAN MIGUEL CO DETENTION CENTE...,Specialty Distributor,Diamond,NMSM,SAN MIGUEL CO DETENTION CENTER,26 NM 283,LAS VEGAS,NM,87701,4,5/14/2021 12:00:00 AM,16
1,EMUSHCO0000002,0,0.864535,1,CommonCents | Montville Police Department | Mo...,Specialty Distributor,CommonCents,Montville Police Department,Montville Police Department,911 Norwich-New London Turnpike,Montville,CT,06382,82,7/13/2023 12:00:00 AM,82
2,EMUSHCO0000003,293,1.000000,2,Chargeback | | IRONWOOD STATE PRISON PHARMACY...,Specialty Distributor,Chargeback,NaN,IRONWOOD STATE PRISON PHARMACY,19005 WILEYS WELL RD,BLYTHE,CA,92225,4,10/6/2022 12:00:00 AM,161
3,EMUSHCO0000004,294,1.000000,3,Chargeback | | HAMILTON COUNTY HEALTH DEPARTM...,Specialty Distributor,Chargeback,NaN,HAMILTON COUNTY HEALTH DEPARTMENT,921 EAST THIRD STREET,CHATTANOOGA,TN,37403,4,8/11/2023 12:00:00 AM,416
4,EMUSHCO0000005,295,1.000000,4,"Truax | | SHERIDAN CTY,DISTRICT #3 | 1008 WAT...",Specialty Distributor,Truax,NaN,"SHERIDAN CTY,DISTRICT #3",1008 WATER STREET,CLEARMONT,WY,82835,1,10/11/2022 2:47:00 PM,1


### Add new column with Record Ids back to original data

In [17]:
## Add the new column back to the original data . The how='left' argument in the merge function ensures that all rows from master_data are retained, and the MCM_ID column is added where applicable.
new_account_data = new_account_data.merge(filtered_new_account_data[['MCM_ID']], left_index=True, right_index=True, how = 'left') #merge function is used with left_index=True and right_index=True to ensure that rows are matched by their index.

In [20]:
# Reorder columns to make MCM ID the first column
cols= ['MCM_ID'] + [col for col in filtered_new_account_data if col != 'MCM_ID']
new_account_data = new_account_data[cols]
 
filtered_new_account_data.head()

,MCM_ID,SpecDistr_Cluster ID,SpecDistr_Link Score,SpecDistr_Name,SpecDistr_Address,SpecDistr_City,SpecDistr_State,SpecDistr_Zip,SpecDistr_Q_ID,SpecDistr_Source,...,SFDC_LastReferencedDate,SFDC_Trade_Partner_Name__c,SFDC_Account_Unique_Id__c,SFDC_Customer_Group__c,SFDC_Shipping_COT__c,SFDC_Ship_To_Number__c,SFDC_RecordType,SFDC_ND_Buying_Company_ID__c,SFDC_ND_Entity_ID__c,SFDC_ND_Shipto_Number__c
0,0000001-20231127,2403,0.973083,"A BETOR,WAY TN",585 SHADY HOLLOW CV,EADS,TN,38028,"Truax | | A BETOR,WAY TN | 585 SHADY HOLLOW C...",Truax,...,NaT,0014x00000EBxcHAAT,AC-0044996,NaN,COMMUNITY BASED ORGANIZATION,NaN,Special Entity Group,NaN,NaN,NaN
1,0000002-20231127,2283,0.946957,"CHEROKEE COUNTY,BOE",1950 SAND ROCK AVE,SAND ROCK,AL,35983,"Truax | | CHEROKEE COUNTY,BOE | 1950 SAND ROC...",Truax,...,NaT,0014x00000EBxQVAA1,AC-0033535,NaN,SCHOOLS / UNIVERSITIES,NaN,Special Entity Group,NaN,NaN,NaN
2,0000003-20231127,3878,0.946957,"A HELPING HAND HEALTH SERVICES, LLC","6401 DOGWOOD ROAD, SUITE 201",WOODLAWN,MD,21207,Chargeback | | A HELPING HAND HEALTH SERVICES...,Chargeback,...,NaT,0014x00000EBxR9AAL,AC-0056417,NaN,HOSPITAL / CLINIC,NaN,Special Entity Group,NaN,NaN,NaN
3,0000004-20231127,54,0.946569,"WYOMING,INDIAN SCHOO",638 BLUE SKY HWY,LANDER,WY,82520,"Truax | | WYOMING,INDIAN SCHOO | 638 BLUE SKY...",Truax,...,NaT,0014x00000EByAGAA1,AC-0010638,NaN,SCHOOLS / UNIVERSITIES,NaN,Special Entity Group,NaN,NaN,NaN
4,0000005-20231127,2032,0.946036,"CITY OF RAMSEY,FIRE",7550 SUNWOOD DRIVE NW,ANOKA,MN,55303,"Truax | | CITY OF RAMSEY,FIRE | 7550 SUNWOOD ...",Truax,...,NaT,0014x00000EBxQVAA1,AC-0030773,NaN,FIRE DEPARTMENTS,NaN,Special Entity Group,NaN,NaN,NaN


## Save the results to excel, append to existing excel file 

In [21]:

# Save the data to excel , append to existing file  --> be sure use that file's name (variable) when writing

with pd.ExcelWriter(raw_input_file, mode= 'a' , engine= 'openpyxl', if_sheet_exists='replace') as writer:
    #filtered_data.to_excel(writer, sheet_name= 'FuzzyMatch_MCM_IDs' , index=False) # use this if you want to write to another sheet separate from the input sheet
    new_account_data.to_excel(writer, sheet_name= '2a. Fuzzy Results' , index= False)

print('Analysis completed and  new accounts saved')

Analysis completed and  new accounts saved


In [10]:
#File locations

input_base_path =r"C:\Users\beste\OneDrive - Qral Group\01 Narcan\12 Ops\20231128_A1\02 Internal Controls\mastering" #update path if needed

raw_input_file = os.path.join(input_base_path, 'Dedupped__SpecDist Mastering Input20231128_175947_Analysis.xlsx') # Set path containing the file with previously analyzed accounts


# Save the data to excel , append to existing file  --> be sure use that file's name (variable) when writing

with pd.ExcelWriter(raw_input_file, mode= 'a' , engine= 'openpyxl', if_sheet_exists='replace') as writer:
    #filtered_data.to_excel(writer, sheet_name= 'FuzzyMatch_MCM_IDs' , index=False) # use this if you want to write to another sheet separate from the input sheet
    new_account_data.to_excel(writer, sheet_name= 'Dedup SpecDistr_MCM_ID_added' , index= False)

print('MCM_ID added to accounts')

MCM_ID added to accounts
